In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
#power provided by one hypha
rho_TAG = 0.18
eTAG = 30e-3 #uJ.um-3 30MJ.L-1
rho_e = rho_TAG*eTAG #uJ.um-3
r_hypha = 3 #um
v_transport = 7 #um.s-1
Ptrunk = np.pi*r_hypha**2*rho_e*v_transport #uW
Ptrunk

1.0687698207512475

In [3]:
phi_TAG = v_transport*rho_TAG*np.pi*r_hypha**2 #um3.s-1
phi_TAG = phi_TAG*3600 #um3.h-1
phi_TAG = phi_TAG/1e15*0.8*1e9 #ug.h-1
phi_TAG

0.10260190279211977

In [5]:
density = 1 #mm.mm-2
density = 100 #mm.cm-2
density = density*1e6 #um.cm-2
volume_density = density * np.pi*r_hypha**2 #um**3.cm-2 fl.cm-2
mass_density = volume_density * 1e-15 #kg.cm-2 
mass_density = mass_density * 1e9 #ug.cm-2
mass_density

2827.4333882308138

***spore loading***

In [4]:
mean_spore_diameter = 60 #um
spore_volume = (mean_spore_diameter/2)**3*4/3*np.pi
rho_TAG = 0.1
e_spore = spore_volume*rho_TAG*eTAG
e_spore #uJ

339.2920065876977

In [7]:
reachable_length = e_spore/e_total_prot_synth
reachable_length

NameError: name 'e_total_prot_synth' is not defined

In [6]:
r_vesicle = 1 #um
e_walking = 100 #ATP.um-1
e_ATP= 1.33e-13 #uJ.ATP-1
Pwalking =v_transport * e_walking*e_ATP #uW
p_transport = rho_TAG*Pwalking*r_hypha**2/(r_vesicle**3) #uW.um-1
L = 1e6 #um
P_transport = L*p_transport #uW
P_transport

8.379e-05

In [8]:
e_total_prot_synth

NameError: name 'e_total_prot_synth' is not defined

In [9]:
L_prot = 400 #AA
n = 3e6 #um-3
Eproteinsynthesis = 4 #ATP.AA-1
e_total_prot_synth = Eproteinsynthesis*e_ATP*L_prot*n*np.pi*r_hypha**2 #uJ.um-1
v_colony_growth = 30000 #um.h-1
v_colony_growth = v_colony_growth/3600 #um.s-1
P_total_protein_synthesis = e_total_prot_synth*v_colony_growth #uW
P_total_protein_synthesis

0.15041945625387926

In [10]:
rho_necessary = P_total_protein_synthesis/(np.pi*r_hypha**2*eTAG*v_transport)

In [11]:
rho_necessary

0.025333333333333326

***The CUE approach***

In [50]:
num_days = 15
r_hypha = 3 #um
quad_mod = quadratic_model()
v_growth = 300/3600 #um.h-1
# density_sat = 1250e-6
density_sat = 1800e-6 #mm.mm-2

# density_sat = 6e-6
init_L = 60000
init_area = np.min(data['area_sep_comp'])
quad_mod.set_alpha(alpha)
quad_mod.set_init_L(init_L)
quad_mod.set_init_area(init_area)

quad_mod.set_density_sat(density_sat)
quad_mod.set_speed_growth(v_growth)
t = np.linspace(0,3600*24*num_days,100)
Ps = quad_mod.get_P_total_protein_synthesis(t)
rhos = Ps/(np.pi*r_hypha**2*eTAG*v_transport)
areas = quad_mod.get_area(t)
radius = np.sqrt(areas / (np.pi/2))/1000 #mm

In [51]:
areas_real = data['area_sep_comp']*10**6
radius_real = np.sqrt(areas_real.astype(float) / (np.pi/2))/1000 #mm
radius_real

544     7.853626
545     7.613929
546     8.382391
547     8.235578
548     8.644636
         ...    
639    32.689706
640    32.935142
641    33.011714
642    33.338653
643    33.370437
Name: area_sep_comp, Length: 100, dtype: float64

In [58]:
fig, ax = plt.subplots()
ax.plot(radius,rhos*100)
ax.set_ylabel('theoretical % trunk hyphal volume occupied with TAG')
ax.set_xlabel('colony radius (mm)')
ax.set_ylim((0,50))
ax.set_xlim((0,40))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 40.0)

In [17]:
class Energetic_model:
    def init(self,params):
        self.params = params
    def set_alpha(self,alpha):
        self.alpha = alpha
    def set_init_L(self,L):
        self.init_L = L
    def set_init_area(self,area):
        self.init_area = area
    def get_L(self,t):
        return(None)
    def get_L_exp(self,t):
        return(self.init_L*np.exp(self.alpha*t))
    def set_speed_growth(self,speed_growth):
        self.v = speed_growth
    def get_v_colony(self,t):
        y = self.get_L(t)
        dydx = np.gradient(y, t)
        return(dydx)
    def get_P_total_protein_synthesis(self,t):
        return(e_total_prot_synth*self.get_v_colony(t))
    def get_P_transport(self,t):
        return(p_transport*self.get_L(t))
    def get_area(self,t):
        return(np.pi/2*(self.v*t)**2+self.init_area)
    def get_area_ring(self,t):
        return(np.pi*(self.v*t))
    
class quadratic_model(Energetic_model):
    def set_density_sat(self,dens_sat):
        self.dens_sat = dens_sat
    def get_L(self,t):
        L_exp = self.get_L_exp(t)
        area = self.get_area(t)
        dens = L_exp/area
        is_above_sat = dens >=self.dens_sat
        return(area * self.dens_sat*is_above_sat+(1-is_above_sat)*self.get_L_exp(t))

class exp_model(Energetic_model):
    def get_L(self,t):
        return(self.get_L_exp(t))

In [15]:
np.min(data['tot_length_study']/data['area_sep_comp'])

NameError: name 'data' is not defined

In [30]:
path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\time_plate_infos.pick"

time_plate_info = pd.read_pickle(path_save)
data = time_plate_info.loc[time_plate_info['Plate']==94]
quad_mod = quadratic_model()
alpha = 1.3/(24*3600)
v_growth = 300/3600
# density_sat = 1250e-6
density_sat = 1800e-6

# density_sat = 6e-6
init_L = 20000
init_area = init_L/(np.max(data['tot_length_study']/data['area_sep_comp'])*1e-6)
quad_mod.set_alpha(alpha)
quad_mod.set_init_L(init_L)
quad_mod.set_init_area(init_area)

quad_mod.set_density_sat(density_sat)
quad_mod.set_speed_growth(v_growth)
exp_mod = exp_model()
exp_mod.set_alpha(alpha)
exp_mod.set_init_L(init_L)
exp_mod.set_speed_growth(v_growth)
exp_mod.set_init_area(init_area)



In [21]:
%matplotlib widget
num_days = 30
t = np.linspace(0,3600*24*num_days,100)
area = quad_mod.get_area(t)
fig, ax = plt.subplots()
ax.plot(area,quad_mod.get_L(t),label = 'quadratic')
ax.plot(area,exp_mod.get_L(t),label='exponential')
ax.scatter(data['area_sep_comp']*10**6,data['tot_length_study'],label='data',s=5)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('area($\mu m^{2}$)')
ax.set_ylabel('Length ($\mu m$)')
ax.vlines(np.max(data['area_sep_comp']*10**6),np.min(quad_mod.get_L(t)),10*np.max(quad_mod.get_L(t)),color="red")
ax.set_ylim((1e5,1e12))
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
%matplotlib widget

fig, ax = plt.subplots()
ax.plot(area,quad_mod.get_L(t)/area,label = 'quadratic')
ax.plot(area,exp_mod.get_L(t)/area,label='exponential')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('area($\mu m^{2}$)')
ax.set_ylabel('density ($\mu m.\mu m^{-2}$)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
%matplotlib widget

fig, ax = plt.subplots()
y = quad_mod.get_P_transport(t)
ax.plot(area,quad_mod.get_P_transport(t),label = 'quadratic')
ax.plot(area,exp_mod.get_P_transport(t),label='exponential')
ax.plot(area,0*t+3,label = "Power provided by one trunk hypha")
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('area($\mu m^{2}$)')
ax.set_ylabel('Power required for transport ($\mu W$)')
ax.vlines(np.max(data['area_sep_comp']*10**6),np.min(y),10*np.max(y),color="red")

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
%matplotlib widget

fig, ax = plt.subplots()
# ax2 = ax.twinx()
y = quad_mod.get_P_total_protein_synthesis(t)
ax.plot(area,0*t+3,label = "Power provided by one trunk hypha")
ax.plot(area,quad_mod.get_P_total_protein_synthesis(t),label = 'power required for protein synthesis (travelling wave)')
ax.plot(area,exp_mod.get_P_total_protein_synthesis(t),label='power required for protein synthesis (exponential)')
ax.plot(area,quad_mod.get_P_transport(t),label = 'power required for molecular motor transport (travelling wave)')
ax.plot(area,exp_mod.get_P_transport(t),label='power required for molecular motor transport (exponential)')
ax.set_yscale('log')
ax.set_xscale('log')
# ax2.set_yscale('log')
# ax2.set_xscale('log')
ax.set_xlabel('area($\mu m^{2}$)')
ax.set_ylabel('Power ($\mu W$)')
# ax.vlines(np.max(data['area_sep_comp']*10**6),np.min(y),10*np.max(y),color="red")
ax.vlines(np.max(data['area_sep_comp']*10**6),10e-6,10e12,color="black",linestyle = "dashed",label="maximum area observed in the experiment")
ax.vlines(8e8,10e-6,10e12,color="red",linestyle = "dashed",label = "point of deviation from the exponential")

ax.legend(fontsize = 6)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …